In [1]:
import pandas as pd
import pyodbc

# -------------------------
# SQL Server connection
# -------------------------
server = '157.119.230.120,4071'
database = 'PowerBI_LISV2'
username = 'sa'
password = 'Y0m@Sql.!.123'

conn_str = f"""
    DRIVER={{ODBC Driver 17 for SQL Server}};
    SERVER={server};
    DATABASE={database};
    UID={username};
    PWD={password};
    TrustServerCertificate=yes;
"""

conn = pyodbc.connect(conn_str)

# -------------------------
# Client list
# -------------------------
x = [
    'AmulManagement', 'CholayilManagement'
]

# -------------------------
# Store summary info
# -------------------------
summary_data = []

# -------------------------
# Fetch data per client
# -------------------------
for client in x:
    query = f"""
        SELECT *,
                AdhocVisibilityReportExportId AS UserKey
        FROM {client}.AdhocVisibilityReportExport
        WHERE Month = '1' AND Year = '2026';
    """

    df = pd.read_sql(query, conn)

    # Clean column names
    df.columns = df.columns.str.strip()

    # Ensure UserKey exists
    if 'UserKey' not in df.columns or df['UserKey'].isna().all():
        df['UserKey'] = (
            df['UserName'].astype(str) + '-' +
            pd.to_datetime(df['Date'], errors='coerce')
              .dt.day.astype(str).str.zfill(2)
        )

    # Create dynamic DataFrame variable
    df_name = f"{client}_Free_Visibility"
    globals()[df_name] = df

    print(f"{df_name} created with {len(df)} rows")

    # Collect summary info
    summary_data.append({
        "ClientName": client,
        "No_of_Rows": len(df)
    })

# -------------------------
# Client-wise summary (ONLY ONCE)
# -------------------------
summary_df = pd.DataFrame(summary_data)
print("\nClient-wise Free Visibility Summary:")
print(summary_df)


C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_40452\603374704.py:46: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


AmulManagement_Free_Visibility created with 2744 rows
CholayilManagement_Free_Visibility created with 1067 rows

Client-wise Free Visibility Summary:
           ClientName  No_of_Rows
0      AmulManagement        2744
1  CholayilManagement        1067


C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_40452\603374704.py:46: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
